<a href="https://colab.research.google.com/github/Adhvik09-oss/Adhvik09-oss/blob/main/SnapCareipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch as nn
import torchvision.datasets as datasets
